# Exploratory data analysis on companies in Glassdoor reviews

In this notebook, I will begin doing some EDA on the Glassdoor Reviews dataset. This dataset contains over 2.5 million reviews on over 5,500 publicly traded companies. I will then create a `companies` DataFrame that contains info on every company reviewed in the dataset. Finally, I will create a `reviews_company_status_title_counts` DataFrame that states how many each times each company-title pair has been reviewed at each company. I will save these last two DataFrames to CSV's for future use.

I will remove reviews that have strange formatting, like improperly written job title. The number of reviews I remove will total up to less than $1\%$, so I won't worry about it too much. 

<b> As a part of using this data from Thinknum, I will not be able to show the data, but I can show my results and analysis.</b>

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 100)

import time

In [ ]:
#import Glassdoor reviews
reviews = pd.read_csv('glassdoor_reviews_2.csv')

In [4]:
print('Number of unique company IDs: {}'.format(reviews.loc[:,'Company Id'].nunique()))

Number of unique company IDs: 5833


In [7]:
print('Number of reviews: {}'.format(reviews.shape[0]))
print('Number of columns: {}'.format(reviews.shape[1]))

print('-'*50)

print('Columns:')

reviews.columns

Number of reviews: 2631927
Number of columns: 35
--------------------------------------------------
Columns:


Index(['Ticker Symbol', 'Entity Name', 'Dataset', 'CUSIP', 'ISIN', 'Unique ID',
       'As Of Date', 'Review Url', 'Logo', 'Author Title', 'Author Location',
       'Author Country', 'Summary', 'Description', 'PROs', 'CONs',
       'Recommends Value', 'Recommends Description', 'Outlook Value',
       'Outlook Description', 'CEO Review Value', 'CEO Review Description',
       'Helpful Count', 'Rating: Overall', 'Rating: Work/Life Balance',
       'Rating: Culture & Values', 'Rating: Career Opportunities',
       'Rating: Comp & Benefits', 'Rating: Senior Management', 'Company Id',
       'Company URL', 'Date Added', 'Date Updated', 'Ticker Sector',
       'Ticker Industry'],
      dtype='object')

In [8]:
reviews.dtypes

Ticker Symbol                    object
Entity Name                      object
Dataset                           int64
CUSIP                            object
ISIN                             object
Unique ID                         int64
As Of Date                       object
Review Url                       object
Logo                             object
Author Title                     object
Author Location                  object
Author Country                   object
Summary                          object
Description                      object
PROs                             object
CONs                             object
Recommends Value                float64
Recommends Description           object
Outlook Value                   float64
Outlook Description              object
CEO Review Value                float64
CEO Review Description           object
Helpful Count                   float64
Rating: Overall                   int64
Rating: Work/Life Balance       float64


## Companies DataFrame

In this section, I will create a DataFrame containing info on every company reviewed. The columns will be:
- `Ticker Symbol`
- `Ticker Sector`
- `Ticker Industry`
- `Company Id`
- `Company URL`
- `company_name`
- `count`: number of times company has been reviewed at company.

I think the rest of the columns are self-explanatory. 

We will use this DataFrame later when we filter our analysis to companies with a minimum number of reviews. Also, we will use the Company ID to distinguish companies later when analyzing employee sentiment.

### Extract company names from URL's for 100 reviews

In [11]:
#extract first 101 reviews
reviews_first_hundred = reviews.loc[:100,:].copy()

reviews_first_hundred.shape

(101, 35)

One column of the reviews DataFrame is the company URL of the reviewed company. An example URL is https://www.glassdoor.com/Overview/Working-at-Genentech-EI_IE274.11,20.htm, where the reviewed company is Genentech. To extract 'Genentech', we will naively just split by '-' and select the second item.

In [12]:
#extract company name from company URL by split
reviews_first_hundred.loc[:,'company_name'] = reviews_first_hundred.loc[:,'Company URL'].apply(lambda x: x.split('-')[2])

In [13]:
for idx in range(101):
    print('Index: {}'.format(idx))
    print('URL: {}'.format(reviews_first_hundred.loc[idx,'Company URL']))
    print('Name: {}'.format(reviews_first_hundred.loc[idx,'company_name']))
    print('-'*50)

Index: 0
URL: https://www.glassdoor.com/Overview/Working-at-Genentech-EI_IE274.11,20.htm
Name: Genentech
--------------------------------------------------
Index: 1
URL: https://www.glassdoor.com/Overview/Working-at-Falabella-EI_IE10976.11,20.htm
Name: Falabella
--------------------------------------------------
Index: 2
URL: https://www.glassdoor.com/Overview/Working-at-Big-W-EI_IE473193.11,16.htm
Name: Big
--------------------------------------------------
Index: 3
URL: https://www.glassdoor.com/Overview/Working-at-WorleyParsons-EI_IE35193.11,24.htm
Name: WorleyParsons
--------------------------------------------------
Index: 4
URL: https://www.glassdoor.com/Overview/Working-at-ExxonMobil-EI_IE237.11,21.htm
Name: ExxonMobil
--------------------------------------------------
Index: 5
URL: https://www.glassdoor.com/Overview/Working-at-Caterpillar-EI_IE137.11,22.htm
Name: Caterpillar
--------------------------------------------------
Index: 6
URL: https://www.glassdoor.com/Overview/Work

We see that we run into an issue for companies with names that are more than 1 word. Thus, on the split by '-', we will select the list items from the second index to the second to last.

In [15]:
reviews_first_hundred.loc[:,'company_name'] = reviews_first_hundred.loc[:,'Company URL'].apply(lambda x: " ".join(x.split('-')[2:-1]))

In [16]:
#check that extraction of company name worked for first 100 reviews
for idx in range(101):
    print('Index: {}'.format(idx))
    print('URL: {}'.format(reviews_first_hundred.loc[idx,'Company URL']))
    print('Name: {}'.format(reviews_first_hundred.loc[idx,'company_name']))
    print('-'*50)

Index: 0
URL: https://www.glassdoor.com/Overview/Working-at-Genentech-EI_IE274.11,20.htm
Name: Genentech
--------------------------------------------------
Index: 1
URL: https://www.glassdoor.com/Overview/Working-at-Falabella-EI_IE10976.11,20.htm
Name: Falabella
--------------------------------------------------
Index: 2
URL: https://www.glassdoor.com/Overview/Working-at-Big-W-EI_IE473193.11,16.htm
Name: Big W
--------------------------------------------------
Index: 3
URL: https://www.glassdoor.com/Overview/Working-at-WorleyParsons-EI_IE35193.11,24.htm
Name: WorleyParsons
--------------------------------------------------
Index: 4
URL: https://www.glassdoor.com/Overview/Working-at-ExxonMobil-EI_IE237.11,21.htm
Name: ExxonMobil
--------------------------------------------------
Index: 5
URL: https://www.glassdoor.com/Overview/Working-at-Caterpillar-EI_IE137.11,22.htm
Name: Caterpillar
--------------------------------------------------
Index: 6
URL: https://www.glassdoor.com/Overview/Wo

### Create companies DataFrame for all reviews

In [17]:
#finds number of distinct companies
reviews.loc[:,'Company URL'].nunique()

5851

In [20]:
#Creates companies DataFrame
companies = reviews.loc[:,['Ticker Symbol',
                           'Ticker Sector', 
                           'Ticker Industry',
                           'Company Id', 
                           'Company URL']].drop_duplicates()

In [22]:
#finds Company ID's are repeated in companies dataframe
companies.loc[:,'Company Id'].value_counts()

3480       3
14916      3
367190     3
3477       3
42406      2
975783     2
14790      2
1985       2
9347       2
10350      2
10648      2
658        2
8347       2
8690       2
3489       2
7330       2
1337673    2
40260      2
13598      2
930001     2
320880     2
11978      2
1190       2
3322       2
234929     2
241146     2
42820      2
733735     1
236210     1
2732       1
          ..
8843       1
9622       1
11671      1
3925       1
14984      1
252482     1
2151       1
17798      1
261509     1
300420     1
992630     1
295        1
7525       1
1382       1
21864      1
9578       1
1390       1
972144     1
4736       1
19831      1
3459       1
17786      1
11046      1
5500       1
7549       1
943486     1
22669      1
42511      1
1410       1
18433      1
Name: Company Id, dtype: int64

In [23]:
#find that there can be multiple compay URL's for same company ID
companies_roche = companies[companies['Company Id']==3480]

for idx in [209,517,49989]:
    print(companies_roche.loc[idx,'Company URL'])

https://www.glassdoor.com/Overview/Working-at-Roche-EI_IE3480.11,16.htm
https://www.glassdoor.com/Overview/Working-at-Roche-Sequencing-Solutions-EI_IE42543.11,37.htm
https://www.glassdoor.com/Overview/Working-at-454-Life-Sciences-EI_IE13757.11,28.htm


In [24]:
companies[companies['Company Id']==14916]

,Ticker Symbol,Ticker Sector,Ticker Industry,Company Id,Company URL
1387,nasdaq:rjet,NaN,Travel & Leisure,14916,https://www.glassdoor.com/Overview/Working-at-...
46859,nasdaq:rjet,NaN,Travel & Leisure,14916,https://www.glassdoor.com/Overview/Working-at-...
2173207,nasdaq:rjet,NaN,Travel & Leisure,14916,https://www.glassdoor.com/Overview/Working-at-...


In [25]:
#23 companies have two url's
#4 companies have three url's
companies.loc[:,'Company Id'].value_counts().value_counts()

1    5806
2      23
3       4
Name: Company Id, dtype: int64

In [26]:
#company ID's with three URL's
three_urls = companies.loc[:,'Company Id'].value_counts().index.tolist()[:4]

#company ID's with two URL's
two_urls = companies.loc[:,'Company Id'].value_counts().index.tolist()[4:4+23]

In [27]:
three_urls

[3480, 14916, 367190, 3477]

In [28]:
#find three URL's corresponding to Caesar's Entertainment (legacy names)
companies[companies['Company Id'] == three_urls[2]].loc[:,'Company URL'].apply(lambda x: print(x))

https://www.glassdoor.com/Overview/Working-at-Caesars-Entertainment-EI_IE367190.11,32.htm
https://www.glassdoor.com/Overview/Working-at-World-Series-of-Poker-WSOP-EI_IE473728.11,37.htm
https://www.glassdoor.com/Overview/Working-at-Rio-All-Suite-Hotel-and-Casino-EI_IE1614.11,41.htm


593        None
45386      None
1249987    None
Name: Company URL, dtype: object

In [29]:
#print URL's for company ID's with multiple URL's

print('Three urls')

print('-'*50)

for company_id in three_urls:
    print('Company ID: {}'.format(company_id))
    companies[companies['Company Id']==company_id].loc[:,'Company URL'].apply(lambda x: print(x))
    print('-'*50)
    
print('-'*50)
print('-'*50)
print('-'*50)
    

print('Two urls')

print('-'*50)
print('-'*50)
print('-'*50)
    
for company_id in two_urls:
    print('Company ID: {}'.format(company_id))
    companies[companies['Company Id']==company_id].loc[:,'Company URL'].apply(lambda x: print(x))
    print('-'*50)
    
#PRA Group is second: 975783
#Union Bank is second: 1985
#Sanofi is second: 9347
#OSRAM is second: 10648
#OneMain Financial is second: 13598
#Uniqlo is second: 320880
#Bob Evans Restaurant is second: 1190

Three urls
--------------------------------------------------
Company ID: 3480
https://www.glassdoor.com/Overview/Working-at-Roche-EI_IE3480.11,16.htm
https://www.glassdoor.com/Overview/Working-at-Roche-Sequencing-Solutions-EI_IE42543.11,37.htm
https://www.glassdoor.com/Overview/Working-at-454-Life-Sciences-EI_IE13757.11,28.htm
--------------------------------------------------
Company ID: 14916
https://www.glassdoor.com/Overview/Working-at-Republic-Airways-EI_IE14916.11,27.htm
https://www.glassdoor.com/Overview/Working-at-Chautauqua-Airlines-EI_IE14353.11,30.htm
https://www.glassdoor.com/Overview/Working-at-Shuttle-America-EI_IE22257.11,26.htm
--------------------------------------------------
Company ID: 367190
https://www.glassdoor.com/Overview/Working-at-Caesars-Entertainment-EI_IE367190.11,32.htm
https://www.glassdoor.com/Overview/Working-at-World-Series-of-Poker-WSOP-EI_IE473728.11,37.htm
https://www.glassdoor.com/Overview/Working-at-Rio-All-Suite-Hotel-and-Casino-EI_IE1614.11,41

In [32]:
#find indices for each company related to each URL (for companies with multiple URL's)
companies[companies['Company Id'].isin(three_urls)].loc[:,'Company Id'].sort_values()

459          3477
46498        3477
212087       3477
209          3480
517          3480
49989        3480
1387        14916
46859       14916
2173207     14916
593        367190
45386      367190
1249987    367190
Name: Company Id, dtype: int64

In [33]:
#drops legacy URL's for companies with three links
indices_to_drop_three_urls = [46498,212087,517,49989,46859,2173207,45386,1249987]

In [34]:
#find indices for reviews of companies with 2 URL's 
companies[companies['Company Id'].isin(two_urls)].loc[:,'Company Id'].sort_values()

5080           658
521            658
33616         1190
3223          1190
1330724       1985
623           1985
644           3322
47587         3322
15213         3489
58893         3489
52091         7330
6364          7330
1288          8347
22087         8347
47165         8690
7559          8690
47611         9347
1517          9347
51557        10350
7546         10350
10692        10648
62224        10648
54421        11978
3908         11978
7070         13598
18747        13598
55741        14790
17194        14790
47816        40260
416          40260
49820        42406
169          42406
25372        42820
54127        42820
51012       234929
9952        234929
46392       241146
11741       241146
45765       320880
2650        320880
8405        930001
58269       930001
47804       975783
5100        975783
46194      1337673
140        1337673
Name: Company Id, dtype: int64

In [35]:
#drop legacy URL's of companies with 2 URL's
#    these are indices where the true URL is listed second
indices_to_drop_two_urls_actually_second = [5100, 623, 1517, 6364, 10692, 7070, 2650, 3223]

In [36]:
companies[companies['Company Id'].isin(two_urls)].loc[:,'Company Id'].sort_values().drop(indices_to_drop_two_urls_actually_second)

5080           658
521            658
33616         1190
1330724       1985
644           3322
47587         3322
15213         3489
58893         3489
52091         7330
1288          8347
22087         8347
47165         8690
7559          8690
47611         9347
51557        10350
7546         10350
62224        10648
54421        11978
3908         11978
18747        13598
55741        14790
17194        14790
47816        40260
416          40260
49820        42406
169          42406
25372        42820
54127        42820
51012       234929
9952        234929
46392       241146
11741       241146
45765       320880
8405        930001
58269       930001
47804       975783
46194      1337673
140        1337673
Name: Company Id, dtype: int64

In [37]:
#drop legacy URL's of companies with 2 URL's
#    these are indices where the true URL is listed first 
indices_to_drop_two_urls_actually_first = [5080, 47587, 58893, 22087, 47165,
                                           51557, 54421, 55741, 47816, 49820, 
                                           54127, 51012, 46392, 58269, 46194]

In [39]:
#indices of rows that have Company URL's that don't contain the exact company name in it

#for example, we should drop "https://www.glassdoor.com/Overview/Working-at-Pearson-Education-Services-EI_IE513386.11,37.htm"
#the current URL is "https://www.glassdoor.com/Overview/Working-at-Pearson-EI_IE3322.11,18.htm"

indices_to_drop = indices_to_drop_three_urls + indices_to_drop_two_urls_actually_second + indices_to_drop_two_urls_actually_first

In [40]:
#drop rows with legacy URL's
companies = companies.drop(indices_to_drop).reset_index()

In [41]:
#now all company ID's have a unique URL attached to them 
#    (hopefully with the correct company name in it)
companies.loc[:,'Company Id'].value_counts().value_counts()

1    5833
Name: Company Id, dtype: int64

In [42]:
companies.loc[:,'company_name'] = companies.loc[:,'Company URL'].apply(lambda x: " ".join(x.split('-')[2:-1]))

In [44]:
companies = companies.drop('index',axis=1)

In [45]:
companies.columns

Index(['Ticker Symbol', 'Ticker Sector', 'Ticker Industry', 'Company Id',
       'Company URL', 'company_name'],
      dtype='object')

We will later add a column to companies that states how many times that company has been reviewed.

## Available jobs at companies

We will now create a DataFrame that states how many times each job title at each company has been reviewed. The columns will be:
- `Company Id`
- `company_name`
- `Job Title`
- `Employee Status`
- `count`

A major task will be dealing with erroneously inputted job titles. For example, ~2,200 Starbucks employees wrote their job title as 'Barista - Starbucks' instead of 'Barista'. I choose to simply ignore all of these reviews, since they add up to less than $1\%$ of my 2.5 million reviews.

The `reviews` DataFrame has one column called `Author Title` which lists positions of reviewers in the format "Employee Type - Job Title".

Let's look at if all of the author titles are of the same format "Employee Type - Job Title" with exactly one dash.

In [51]:
#see how many dashes are in each author title
#    note that vast majority of job titles (like 'Engineer') have no dashes
reviews.loc[:,'title_length'] = reviews.loc[:,'Author Title'].apply(lambda x: len(x.split(' - ')))

In [52]:
reviews.loc[:,'title_length'].value_counts()

2    2615691
3      16131
4         73
1         30
5          2
Name: title_length, dtype: int64

In [53]:
#look at reviews with only one dash
reviews[reviews['title_length'] == 1].loc[:,'Author Title'].value_counts()

Current Employee -      19
Former Employee -        9
Current Contractor -     2
Name: Author Title, dtype: int64

We see that these author titles with only one dash simply didn't inclue a job title. Since these form such a small proportion of the reviews (<<1$\%$), we choose to simply drop these reviews.

In [54]:
reviews = reviews[reviews['title_length'] > 1]

Let's now look at the unconventional reviews that aren't in the simple format of having one dash: "Employee Status -  Job Position".

In [55]:
reviews[reviews['title_length'] > 2].loc[:,'Author Title'].value_counts()

Current Employee - Barista - Starbucks                                                   1263
Former Employee - Barista - Starbucks                                                    1112
Current Employee - Associate - Projects                                                   762
Former Intern - Intern - Hourly                                                           710
Current Intern - Intern - Hourly                                                          486
Former Employee - Intern - Hourly                                                         463
Current Employee - Intern - Hourly                                                        312
Current Employee - Vice President - Technology                                            252
Former Employee - Associate - Projects                                                    252
Current Employee - Senior Associate - Projects                                            226
Former Employee - Guest Service Team Member - Cashier       

There are some common issues with the reviews that have at least 2 dashes in "Author Title." For one thing, many of these people mistakenly write when or where they work. Note that these recommendations consists of less than $1\%$ of all the reviews in the dataset. We choose to simply remove them.

The reason is that it's a little tricky to extract and change these- I tried! But also let's think about our goal now. We want to find all of the available jobs at companies. Barista is surely already represented in reviews for Starbucks. Similarly, Associate, Intern, and Vice President should also be represented at those companies for which the bad forms of them are written. 

In [56]:
reviews = reviews[reviews['title_length'] == 2]

## Find available jobs at each company

The way we will store jobs is in a new column `available_jobs` in the "Companies" DataFrame. This will be the set of all of the jobs that have had reviews at the row's company.

In [61]:
companies.head()

,Ticker Symbol,Ticker Sector,Ticker Industry,Company Id,Company URL,company_name
0,vtx:rog,Health Care,Pharmaceuticals & Biotechnology,274,https://www.glassdoor.com/Overview/Working-at-...,Genentech
1,bcs:falabella,NaN,NaN,10976,https://www.glassdoor.com/Overview/Working-at-...,Falabella
2,asx:wow,Consumer Services,Food & Drug Retailers,473193,https://www.glassdoor.com/Overview/Working-at-...,Big W
3,asx:wor,NaN,NaN,35193,https://www.glassdoor.com/Overview/Working-at-...,WorleyParsons
4,nyse:xom,Oil & Gas,Oil & Gas Producers,237,https://www.glassdoor.com/Overview/Working-at-...,ExxonMobil


In [62]:
#initialize column 'available_jobs' as empty set
companies.loc[:,'available_jobs'] = pd.Series([set() for idx in range(companies.shape[0])])

In [63]:
#extracts 'Employee Status' and 'Job Title' from 'Author Title'

reviews.loc[:,'Employee Status'] = reviews.loc[:,'Author Title'].apply(lambda x: x.split(' - ')[0])
reviews.loc[:,'Job Title'] = reviews.loc[:,'Author Title'].apply(lambda x: x.split(' - ')[1])

In [64]:
#find all unique pairs Company Id-Job Title
company_job = reviews.copy().loc[:, ['Company Id','Job Title']].drop_duplicates().reset_index(drop=True)

In [65]:
#number of unique pairs is one fifth number of reviews
print(company_job.shape)

company_job.head()

(535356, 2)


,Company Id,Job Title
0,274,Anonymous Contractor
1,10976,Senior Engineer
2,473193,Anonymous Employee
3,35193,Anonymous Employee
4,237,I T Analyst


In [66]:
company_available_jobs = pd.Series(data=[set() for idx in range(companies.shape[0])], index=companies.loc[:,'Company Id'])

In [67]:
start_time = time.time()

company_job.apply(lambda row: company_available_jobs[row['Company Id']].add(row['Job Title']), axis=1)
    
print('Took ' + str(time.time()-start_time) + ' seconds.')

Took 22.18036985397339 seconds.


1000

In [68]:
company_available_jobs.head(30)

Company Id
274        {Project and Process Engineering, Assistant Sc...
10976      {Senior Engineer, Digital Content Producer, IT...
473193     {Presentation Associate, Fill Associate, Onlin...
35193      {Services, CAD Designer, Document Control Spec...
237        {Electrician, Store, Construction Superintende...
137        {Process Support, Short Term Disability Repres...
1033056    {Administrative Assistant, Human Resources, Of...
6826       {R/C, Correction Officer, Administrative Assis...
15057      {Order Entry Specialist, Staff Engineer, Techn...
17798      {Loan Processor Associate, Servicing Systems A...
16559      {Securitas Security Guard, Security Officer/Co...
2800       {Credit Risk Analyst, Off Cycle, Core Strategi...
1737       {QA Automation Engineer, Technician Support An...
599        {R&D Chemist, Store, Lead Developer, Office Ma...
1202       {Financial Consultant, Wire Specialist, Compli...
404        {Service Mechanic, Electrician, Senior Electro...
1347599    {M

In [69]:
total_jobs_companies = company_available_jobs.apply(lambda x: len(x)).sum()

#number of jobs in lists of company available jobs agrees 
#    with the number of distinct jobs in company_job
assert(total_jobs_companies == company_job.shape[0])

In [75]:
#find erroneous employee statuses
set(reviews.loc[:,'Employee Status'])

{'Current Apprentice',
 'Current Contractor',
 'Current Employee',
 'Current Freelancer',
 'Current Intern',
 'Current PhD',
 'Current Trainee',
 'Former Apprentice',
 'Former Contractor',
 'Former Employee',
 'Former Freelancer',
 'Former Intern',
 'Former PhD',
 'Former Trainee',
 'module.emp-review.current-',
 'module.emp-review.former-'}

In [76]:
reviews = reviews[reviews['Employee Status'] != 'module.emp-review.current-'] #remove 4 reviews

In [77]:
reviews = reviews[reviews['Employee Status'] != 'module.emp-review.former-'] #remove 6 reviews

In [78]:
reviews.loc[:,'current_or_former'] = reviews.loc[:,'Employee Status'].apply(lambda x: x.split(' ')[0])

In [81]:
#DataFrame of all triples of company ID, employee status, job title
reviews_company_status_title = reviews.copy().loc[:,['Company Id', 'Employee Status', 'Job Title']]

In [82]:
reviews_company_status_title.head(10)

,Company Id,Employee Status,Job Title
0,274,Current Contractor,Anonymous Contractor
1,10976,Current Employee,Senior Engineer
2,473193,Current Employee,Anonymous Employee
3,35193,Former Employee,Anonymous Employee
4,237,Former Employee,I T Analyst
5,137,Current Employee,Welder/Fabricator
6,1033056,Current Employee,Quality
7,6826,Former Employee,Administrative Clerk
8,15057,Current Employee,Anonymous Employee
9,17798,Former Employee,Anonymous Employee


In [83]:
reviews_company_status_title_counts = reviews_company_status_title.groupby(by=['Company Id', 'Job Title', 'Employee Status'])['Employee Status'].count()


In [84]:
reviews_company_status_title_counts = pd.DataFrame({'count': reviews_company_status_title.groupby(['Company Id', 'Job Title', 'Employee Status']).size()}).reset_index()


In [85]:
#how many times each job title has been reviewed at each company
reviews_company_status_title_counts.head()

,Company Id,Job Title,Employee Status,count
0,4,A&P,Current Employee,1
1,4,A&P Lead,Current Employee,1
2,4,A&P Lead Mechanic,Current Employee,1
3,4,A&P Mechanic,Current Employee,1
4,4,A&P Mechanic,Former Contractor,2


In [86]:
reviews_company_status_title_counts = reviews_company_status_title_counts.merge(companies.loc[:,['Company Id','company_name']], on='Company Id')

In [88]:
reviews_company_count = reviews_company_status_title_counts.loc[:,['Company Id', 'count']]

In [89]:
#number of reviews for each company
reviews_company_total = reviews_company_status_title_counts.groupby('Company Id')['count'].count().reset_index()

In [91]:
#company Id 1071540 no longer in set of reviews (due to dropping rows)
set(companies.loc[:,'Company Id']) - set(reviews_company_total.loc[:, 'Company Id']) 

{1071540}

In [92]:
#remove row with company Id 1071540
companies = companies[companies['Company Id'] != 1071540]

In [93]:
companies = companies.drop('available_jobs', axis=1)

In [94]:
#add column that states how many times each company has been reviewed
companies = companies.merge(reviews_company_total, on='Company Id')

In [95]:
companies

,Ticker Symbol,Ticker Sector,Ticker Industry,Company Id,Company URL,company_name,count
0,vtx:rog,Health Care,Pharmaceuticals & Biotechnology,274,https://www.glassdoor.com/Overview/Working-at-...,Genentech,609
1,bcs:falabella,NaN,NaN,10976,https://www.glassdoor.com/Overview/Working-at-...,Falabella,9
2,asx:wow,Consumer Services,Food & Drug Retailers,473193,https://www.glassdoor.com/Overview/Working-at-...,Big W,70
3,asx:wor,NaN,NaN,35193,https://www.glassdoor.com/Overview/Working-at-...,WorleyParsons,379
4,nyse:xom,Oil & Gas,Oil & Gas Producers,237,https://www.glassdoor.com/Overview/Working-at-...,ExxonMobil,845
5,nyse:cat,Industrials,Industrial Engineering,137,https://www.glassdoor.com/Overview/Working-at-...,Caterpillar,952
6,nyse:wrk,Industrials,General Industrials,1033056,https://www.glassdoor.com/Overview/Working-at-...,WestRock,275
7,nyse:cxw,Financials,General Financial,6826,https://www.glassdoor.com/Overview/Working-at-...,Corrections Corporation of America,83
8,lon:mcro,Technology,Software & Computer Services,15057,https://www.glassdoor.com/Overview/Working-at-...,Micro Focus,204
9,nyse:phh,Industrials,Industrial Transportation,17798,https://www.glassdoor.com/Overview/Working-at-...,PHH Mortgage,140


In [96]:
#find percentage of companies that have at most each number of reviews
companies_percentages = (companies.loc[:,'count'].value_counts()*100/5832)

companies_percentages.cumsum()

1         4.972565
2         9.087791
3        13.185871
4        16.923868
5        20.541838
6        23.593964
7        26.217421
8        28.497942
9        30.401235
10       32.270233
11       33.779150
13       35.270919
12       36.728395
14       38.185871
15       39.283265
17       40.346365
19       41.409465
16       42.455418
18       43.432785
28       44.290123
32       45.113169
20       45.919067
22       46.724966
24       47.496571
23       48.216735
25       48.936900
26       49.622771
31       50.308642
21       50.925926
29       51.543210
           ...    
610      99.502743
614      99.519890
326      99.537037
318      99.554184
314      99.571331
290      99.588477
1185     99.605624
1201     99.622771
1253     99.639918
1257     99.657064
1373     99.674211
1425     99.691358
1477     99.708505
1589     99.725652
1919     99.742798
1693     99.759945
1729     99.777092
1913     99.794239
2025     99.811385
2045     99.828532
4116     99.845679
2071     99.

In [98]:
reviews_company_status_title_counts = reviews_company_status_title_counts.loc[:,['Company Id', 'company_name', 'Job Title', 'Employee Status', 'count']]

In [99]:
reviews_company_status_title_counts

,Company Id,company_name,Job Title,Employee Status,count
0,4,AAR,A&P,Current Employee,1
1,4,AAR,A&P Lead,Current Employee,1
2,4,AAR,A&P Lead Mechanic,Current Employee,1
3,4,AAR,A&P Mechanic,Current Employee,1
4,4,AAR,A&P Mechanic,Former Contractor,2
5,4,AAR,A&P Mechanic,Former Employee,5
6,4,AAR,A&P Technician,Current Employee,3
7,4,AAR,AP Mechanic,Former Employee,1
8,4,AAR,Accountant,Current Employee,1
9,4,AAR,Administrative,Current Employee,1


In [100]:
reviews_company_status_title_counts.to_csv('companies_jobs_counts.csv')

In [101]:
#cumulative percentages of how many times company-job title-count triples have been reviewed
total_reviews = reviews_company_status_title_counts.loc[:,'count'].value_counts().sum()

reviews_company_status_title_counts.loc[:,'count'].value_counts().cumsum()/total_reviews*100

1        75.599843
2        85.898896
3        89.805845
4        91.993890
5        93.341366
6        94.312981
7        95.007516
8        95.552807
9        95.982800
10       96.334655
11       96.618922
12       96.860679
13       97.072923
14       97.259019
15       97.417285
16       97.552767
17       97.670205
18       97.782749
19       97.880308
20       97.972362
21       98.056465
22       98.133075
23       98.201121
24       98.262440
25       98.321159
26       98.378960
27       98.430339
28       98.480342
29       98.523923
31       98.567197
           ...    
2197     99.995565
2872     99.995718
824      99.995871
309      99.996024
663      99.996177
3222     99.996330
4233     99.996483
650      99.996636
1865     99.996789
843      99.996942
635      99.997095
2173     99.997248
636      99.997400
1148     99.997553
859      99.997706
2394     99.997859
638      99.998012
2174     99.998165
3419     99.998318
2904     99.998471
1152     99.998624
641      99.

In [106]:
companies.to_csv('reviewed_companies.csv')